In [1]:
from __future__ import print_function, division
import nilmtk
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from matplotlib import ticker
import numpy as np

In [91]:
from lasagne.layers import (InputLayer, DenseLayer, ReshapeLayer,
                            DimshuffleLayer, Conv1DLayer)
from lasagne.nonlinearities import rectify, tanh

from neuralnilm.data.loadactivations import load_nilmtk_activations
from neuralnilm.data.syntheticaggregatesource import SyntheticAggregateSource
from neuralnilm.data.realaggregatesource import RealAggregateSource
from neuralnilm.data.stridesource import StrideSource
from neuralnilm.data.datapipeline import DataPipeline
from neuralnilm.data.processing import DivideBy, IndependentlyCenter
from neuralnilm.net import Net, build_net
from neuralnilm.trainer import Trainer
from neuralnilm.metrics import Metrics
from neuralnilm.utils import select_windows, filter_activations, configure_logger
from neuralnilm.layers import BLSTMLayer

In [3]:
configure_logger()

In [36]:
NILMTK_FILENAME = '/data/mine/vadeec/merged/ukdale.h5'
SAMPLE_PERIOD = 6
STRIDE = 16
APPLIANCES = [
    'kettle', 'microwave', 'washing machine', 'dish washer', 'fridge']
WINDOWS = {
    'train': {
        1: ("2013-04-12", "2013-06-01")
    },
    'unseen_activations_of_seen_appliances': {
        1: ("2014-12-16", "2015-01-02")
    },
    'unseen_appliances': {
        5: ("2014-09-01", None)
    }
}

In [37]:
def get_pipeline(target_appliance, activations):
    num_seq_per_batch = 64
    if target_appliance == 'kettle':
        seq_length = 128
        max_appliance_power = 3100
    elif target_appliance == 'microwave':
        seq_length = 288
        max_appliance_power = 3000
    elif target_appliance == 'washing machine':
        seq_length = 1024
        max_appliance_power = 2500
    elif target_appliance == 'fridge':
        seq_length = 512
        max_appliance_power = 300
    elif target_appliance == 'dish washer':
        seq_length = 1024 + 512
        max_appliance_power = 2500

    real_agg_source = RealAggregateSource(
        activations=activations,
        target_appliance=target_appliance,
        seq_length=seq_length,
        filename=NILMTK_FILENAME,
        windows=WINDOWS,
        sample_period=SAMPLE_PERIOD
    )

    input_std = 374.43884277
    pipeline = DataPipeline(
        [real_agg_source],
        num_seq_per_batch=num_seq_per_batch,
        input_processing=[DivideBy(input_std), IndependentlyCenter()],
        target_processing=[DivideBy(max_appliance_power)]
    )

    return pipeline

In [90]:
def get_params_filename(appliance, arch):
    if arch == 'rnn':
        experiment = 'e566'
    else:
        experiment = 'e567'
    full_exp_name = experiment + '_' + appliance + '_' + arch
    PATH = '/storage/experiments/neuralnilm/figures/'
    return PATH + full_exp_name + '/' + full_exp_name + '.hdf5'

In [17]:
def get_ae(batch, appliance):
    NUM_FILTERS = 8
    input_shape = batch.input.shape
    target_shape = input_shape
    seq_length = input_shape[1]
    output_layer = build_net(
        input_shape=input_shape,
        layers=[
            
            {
                'type': DimshuffleLayer,
                'pattern': (0, 2, 1)  # (batch, features, time)
            },
            {
                'type': Conv1DLayer,  # convolve over the time axis
                'num_filters': NUM_FILTERS,
                'filter_size': 4,
                'stride': 1,
                'nonlinearity': None,
                'pad': 'valid'
            },
            {
                'type': DimshuffleLayer,
                'pattern': (0, 2, 1)  # back to (batch, time, features)
            },
            {
                'type': DenseLayer,
                'num_units': (seq_length - 3) * NUM_FILTERS,
                'nonlinearity': rectify
            },
            {
                'type': DenseLayer,
                'num_units': 128,
                'nonlinearity': rectify
            },            
            {
                'type': DenseLayer,
                'num_units': (seq_length - 3) * NUM_FILTERS,
                'nonlinearity': rectify
            },            
            {
                'type': ReshapeLayer,
                'shape': (-1, (seq_length - 3), NUM_FILTERS)
            },
            {
                'type': DimshuffleLayer,
                'pattern': (0, 2, 1)  # (batch, features, time)
            },
            {   # DeConv
                'type': Conv1DLayer,
                'num_filters': 1,
                'filter_size': 4,
                'stride': 1,
                'nonlinearity': None,
                'pad': 'full'
            },
            {
                'type': DimshuffleLayer,
                'pattern': (0, 2, 1)  # back to (batch, time, features)
            }
        ]
    )

    net = Net(output_layer)
    net.load_params(get_params_filename(appliance=appliance, arch='ae'), 100000)
    return net

In [100]:
def get_rnn(batch, appliance):
    input_shape = batch.input.shape
    target_shape = input_shape
    seq_length = input_shape[1]
    output_layer = build_net(
        input_shape=input_shape,
        layers=[
            {
                'type': DimshuffleLayer,
                'pattern': (0, 2, 1)  # (batch, features, time)
            },
            {
                'type': Conv1DLayer,  # convolve over the time axis
                'num_filters': 16,
                'filter_size': 4,
                'stride': 1,
                'nonlinearity': None,
                'pad': 'same'
            },
            {
                'type': DimshuffleLayer,
                'pattern': (0, 2, 1)  # back to (batch, time, features)
            },
            {
                'type': BLSTMLayer,
                'num_units': 128,
                'merge_mode': 'concatenate',
                'grad_clipping': 10.0,
                'gradient_steps': 500
            },
            {
                'type': BLSTMLayer,
                'num_units': 256,
                'merge_mode': 'concatenate',
                'grad_clipping': 10.0,
                'gradient_steps': 500
            },
            {
                'type': ReshapeLayer,
                'shape': (input_shape[0] * input_shape[1], 512)
            },
            {
                'type': DenseLayer,
                'num_units': 128,
                'nonlinearity': tanh
            },
            {
                'type': DenseLayer,
                'num_units': 1,
                'nonlinearity': None
            },
            {
                'type': ReshapeLayer,
                'shape': target_shape
            }
        ]
    )

    net = Net(output_layer)
    net.load_params(get_params_filename(appliance=appliance, arch='rnn'), 10000)
    return net

In [38]:
activations = load_nilmtk_activations(
    appliances=APPLIANCES,
    filename=NILMTK_FILENAME,
    sample_period=SAMPLE_PERIOD,
    windows=WINDOWS
)

INFO:neuralnilm.data.loadactivations:Loading NILMTK activations...
INFO:neuralnilm.data.loadactivations:Loading kettle for UK-DALE_building_1...
INFO:neuralnilm.data.loadactivations:Loaded 214 kettle activations from UK-DALE_building_1.
INFO:neuralnilm.data.loadactivations:Loading microwave for UK-DALE_building_1...
INFO:neuralnilm.data.loadactivations:Loaded 180 microwave activations from UK-DALE_building_1.
INFO:neuralnilm.data.loadactivations:Loading washing machine for UK-DALE_building_1...
INFO:neuralnilm.data.loadactivations:Loaded 32 washing machine activations from UK-DALE_building_1.
INFO:neuralnilm.data.loadactivations:Loading dish washer for UK-DALE_building_1...
INFO:neuralnilm.data.loadactivations:Loaded 13 dish washer activations from UK-DALE_building_1.
INFO:neuralnilm.data.loadactivations:Loading fridge for UK-DALE_building_1...
INFO:neuralnilm.data.loadactivations:Loaded 1246 fridge activations from UK-DALE_building_1.
INFO:neuralnilm.data.loadactivations:Loading kettl

In [39]:
APPLIANCE = 'fridge'
pipeline = get_pipeline(APPLIANCE, activations)

INFO:neuralnilm.data.source:------------- INITIALISING RealAggregateSource --------------
INFO:neuralnilm.data.realaggregatesource:Loading NILMTK mains...
INFO:neuralnilm.data.realaggregatesource:Loading mains for UK-DALE_building_1...
INFO:neuralnilm.data.realaggregatesource:Loaded mains data from building UK-DALE_building_1 for fold train from 2013-04-12 00:00:00+01:00 to 2013-05-31 23:59:54+01:00.
INFO:neuralnilm.data.realaggregatesource:Loading mains for UK-DALE_building_5...
INFO:neuralnilm.data.realaggregatesource:Loaded mains data from building UK-DALE_building_5 for fold unseen_appliances from 2014-09-01 00:00:00+01:00 to 2014-11-13 20:35:24+00:00.
INFO:neuralnilm.data.realaggregatesource:Loading mains for UK-DALE_building_1...
INFO:neuralnilm.data.realaggregatesource:Loaded mains data from building UK-DALE_building_1 for fold unseen_activations_of_seen_appliances from 2014-12-16 00:00:00+00:00 to 2015-01-01 23:59:54+00:00.
INFO:neuralnilm.data.realaggregatesource:Done loading 

In [12]:
batch = pipeline.get_batch()

In [18]:
net = get_ae(batch, APPLIANCE)

INFO:neuralnilm.net:Loading params from /storage/experiments/neuralnilm/figures/e567_fridge_ae/e567_fridge_ae.hdf5...
INFO:neuralnilm.net:Done loading params from /storage/experiments/neuralnilm/figures/e567_fridge_ae/e567_fridge_ae.hdf5.


In [40]:
batch = pipeline.get_batch(fold='unseen_activations_of_seen_appliances')

In [41]:
output = net.deterministic_output_func(batch.after_processing.input)

In [101]:
rnn = get_rnn(batch, APPLIANCE)

INFO:neuralnilm.net:Loading params from /storage/experiments/neuralnilm/figures/e566_fridge_rnn/e566_fridge_rnn.hdf5...
INFO:neuralnilm.net:Done loading params from /storage/experiments/neuralnilm/figures/e566_fridge_rnn/e566_fridge_rnn.hdf5.


In [103]:
rnn_output = rnn.deterministic_output_func(batch.after_processing.input)

INFO:neuralnilm.net:Compiling deterministic output function...
/home/dk3810/workspace/python/Theano/theano/scan_module/scan.py:1019: Warning: In the strict mode, all neccessary shared variables must be passed as a part of non_sequences
  'must be passed as a part of non_sequences', Warning)
/home/dk3810/workspace/python/Theano/theano/gof/cmodule.py:324: RuntimeWarning: numpy.ndarray size changed, may indicate binary incompatibility
  rval = __import__(module_name, {}, {}, [module_name])
INFO:neuralnilm.net:Done compiling deterministic output function.


In [156]:
SEQ_I = 11
# 11, 18 (defrost), 30, 32 (very impressive) are good. 
# 27 is good false positive.  48 is good for AE, not for RNN
# 54 is nice clear activation, good for both nets
fig, axes = plt.subplots(2, sharex=True)
axes[0].plot(output[SEQ_I, :, 0], label='AE')
axes[0].plot(rnn_output[SEQ_I, :, 0], label='RNN')
axes[0].plot(batch.target[SEQ_I, :, 0], label='Target')
axes[0].legend()
axes[1].plot(batch.input[SEQ_I, :, 0])
plt.show()

In [157]:
np.save('fridge_input.npy', batch.input)

In [158]:
np.save('fridge_target.npy', batch.target)